In [1]:
# 残差神经网络。
'''
准备数据阶段。
'''
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

train = pd.read_csv('datasets/fashionmnist/fashion-mnist_train.csv')
test = pd.read_csv('datasets/fashionmnist/fashion-mnist_test.csv')

y_train = train['label'].values
y_test = test['label'].values

X_train = train.drop('label', axis=1)
X_test = test.drop('label', axis=1)

ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

train_samples, n_features = np.shape(X_train)
test_samples = len(X_test)

/Users/michael/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/michael/anaconda/lib/python2.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/michael/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:19: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.


In [2]:
'''
构建卷积神经网络阶段（Construction Phase)。
'''
import tensorflow as tf

batch_size = 1024
n_batches = train_samples / batch_size
n_epochs = 50
learning_rate = 1e-2


X = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.int32)

X_images = tf.reshape(X, [-1, 28, 28, 1])

#conv_0 28 * 28
conv_0 = tf.layers.conv2d(inputs=X_images, filters=16, kernel_size=3, strides=1, padding='SAME', activation=tf.nn.relu, name='C0')

#resnet_1 14 * 14

conv_1 = tf.layers.conv2d(inputs=conv_0, filters=32, kernel_size=3, strides=2, padding='SAME', activation=tf.nn.relu, name='C1')

conv_0_down = tf.layers.conv2d(inputs=conv_0, filters=32, kernel_size=1, strides=2, padding='SAME', activation=tf.nn.relu, name='C0_down')

res_1 = conv_1 + conv_0_down

#resnet_2 7 * 7 

conv_2 = tf.layers.conv2d(inputs=res_1, filters=64, kernel_size=3, strides=2, padding='SAME', activation=tf.nn.relu, name='C2')

conv_1_down = tf.layers.conv2d(inputs=res_1, filters=64, kernel_size=1, strides=2, padding='SAME', activation=tf.nn.relu, name='C1_down')

res_2 = conv_2 + conv_1_down


#fc_1
res_2_flat = tf.reshape(res_2, [-1, 64 * 7 * 7])

fc_1 = tf.layers.dense(inputs=res_2_flat, units=100, activation=tf.nn.relu, name='FC_1')

logits = tf.layers.dense(inputs=fc_1, units=10, activation=None, name='FC_2')

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)

loss = tf.reduce_mean(xentropy, name='loss')

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)

accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))


/Users/michael/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    for iteration in range(n_epochs):
        for i in range(n_batches):
            np.random.seed(iteration * n_batches + i)
            indices = np.random.randint(train_samples, size=batch_size)
            X_batch = X_train[indices]
            y_batch = y_train[indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        print 'Iteration %d, loss = %f' % (iteration + 1, loss.eval(feed_dict = {X: X_train, y: y_train}))
    print '分类准确率为：%f' % accuracy.eval(feed_dict = {X: X_test, y: y_test})

Iteration 1, loss = 0.408027
Iteration 2, loss = 0.318039
Iteration 3, loss = 0.282976
Iteration 4, loss = 0.259912
Iteration 5, loss = 0.233980
Iteration 6, loss = 0.219634
Iteration 7, loss = 0.201804
Iteration 8, loss = 0.185846
Iteration 9, loss = 0.170002
Iteration 10, loss = 0.161863
Iteration 11, loss = 0.151641
Iteration 12, loss = 0.132036
Iteration 13, loss = 0.135684
Iteration 14, loss = 0.126233
Iteration 15, loss = 0.118723
Iteration 16, loss = 0.112496
Iteration 17, loss = 0.103044
Iteration 18, loss = 0.101096
Iteration 19, loss = 0.079763
Iteration 20, loss = 0.084523
Iteration 21, loss = 0.084178
Iteration 22, loss = 0.074242
Iteration 23, loss = 0.066431
Iteration 24, loss = 0.066408
Iteration 25, loss = 0.065242
Iteration 26, loss = 0.060844
Iteration 27, loss = 0.053606
Iteration 28, loss = 0.058177
Iteration 29, loss = 0.065722
Iteration 30, loss = 0.050259
Iteration 31, loss = 0.049816
Iteration 32, loss = 0.054844
Iteration 33, loss = 0.058168
Iteration 34, loss 